In [29]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV




TODO:
Preprocessing:
Perform initial data preparation by converting the 'TotalCharges' column to numeric values and filling missing values with 0.
Convert the 'Churn' column to binary values, where 'No' is mapped to 0 and 'Yes' is mapped to 1.<br>
Split the data into an 80-20 train-test split with a random state of “1”.<br>
Select these features:  
categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Contract', 'PaperlessBilling', 'PaymentMethod']
numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']<br>
Feature engineering:
The numerical features should be scaled using StandardScaler, convert the output back to a dataframe and put back the column names.
The categorical features are one-hot encoded using OneHotEncoder(set sparse_output to false), convert the output back to a dataframe and put back the column names.<br>
Combine scaled numerical and one-hot encoded categorical features into train and test set dataframes (use pd.concat)<br>
Use scikit learn to train a random forest and extra trees classifier, and use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set. Answer the following questions:

In [2]:
df = pd.read_csv("Customer_churn.csv")

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Data Preprocessing

In [5]:
# Convert 'TotalCharges' to numeric and fill missing values with 0
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(0, inplace=True)

In [6]:
# Convert 'Churn' to binary values (0 for 'No' and 1 for 'Yes')
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})


In [7]:
categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Contract', 'PaperlessBilling', 'PaymentMethod'] 
numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [8]:
# Split the data into an 80-20 train-test split with a random state of 1
X = df[categorical + numerical]
y = df['Churn']


In [9]:
X.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
tenure              0
MonthlyCharges      0
TotalCharges        0
dtype: int64

### Feature Engineering

In [10]:
# Scale the numerical features using StandardScaler
scaler = StandardScaler()
X[numerical] = scaler.fit_transform(X[numerical])

C:\Users\user\AppData\Local\Temp\ipykernel_15768\3710506873.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical] = scaler.fit_transform(X[numerical])


In [11]:
X.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,tenure,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,-1.277445,-1.160323,-0.992611
1,Male,0,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,0.066327,-0.259629,-0.172165
2,Male,0,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,-1.236724,-0.362660,-0.958066
3,Male,0,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),0.514251,-0.746535,-0.193672
4,Female,0,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,-1.236724,0.197365,-0.938874


In [12]:
# One-Hot Encode the categorical features
encoder = OneHotEncoder(sparse=False)
X_encoded = encoder.fit_transform(X[categorical])

# Convert the encoded features back to dataframes with column names
X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical))

In [14]:
# Combine scaled numerical and one-hot encoded categorical features
X_final = pd.concat([X[numerical], X_encoded], axis=1)


In [15]:
X_final.isna().sum()

tenure                                     0
MonthlyCharges                             0
TotalCharges                               0
gender_Female                              0
gender_Male                                0
SeniorCitizen_0                            0
SeniorCitizen_1                            0
Partner_No                                 0
Partner_Yes                                0
Dependents_No                              0
Dependents_Yes                             0
PhoneService_No                            0
PhoneService_Yes                           0
MultipleLines_No                           0
MultipleLines_No phone service             0
MultipleLines_Yes                          0
InternetService_DSL                        0
InternetService_Fiber optic                0
InternetService_No                         0
OnlineSecurity_No                          0
OnlineSecurity_No internet service         0
OnlineSecurity_Yes                         0
OnlineBack

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=1)

### Model Building

In [17]:
# Train Random Forest and Extra Trees classifiers
rf_model = RandomForestClassifier(random_state=1)
et_model = ExtraTreesClassifier(random_state=1)
rf_model.fit(X_train, y_train)
et_model.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1)

In [20]:
 #Train XGBoost and LightGBM models
xgb_model = XGBClassifier(random_state=1)
lgbm_model = LGBMClassifier(random_state=1)
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785


LGBMClassifier(random_state=1)

14.) What is the accuracy on the test set using the random forest classifier?







In [25]:
rf_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, rf_pred)
print(accuracy)

0.7906316536550745


15.) What is the accuracy on the test set using the xgb classifier

In [26]:
xgb_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, xgb_pred)
print(accuracy)

0.7877927608232789


16.) What is the accuracy on the test set using the lgbm classifier

In [27]:
lgbm_pred = lgbm_model.predict(X_test)
accuracy = accuracy_score(y_test, lgbm_pred)
print(accuracy)

0.8133427963094393


In [39]:
et_pred = et_model.predict(X_test)
accuracy = accuracy_score(y_test, et_pred)
print(accuracy)

0.7700496806245565


17.) To improve the Extra Trees Classifier, you will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV). 

n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

In [32]:
# Define the hyperparameter grid
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'max_features': max_features
}

# Create the RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    et_model, hyperparameter_grid, cv=5, n_iter=10, scoring='accuracy',
    n_jobs=-1, verbose=1, random_state=1
)

# Perform the randomized search on your data
random_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", random_search.best_params_)






Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'n_estimators': 1000, 'min_samples_split': 9, 'min_samples_leaf': 8, 'max_features': 'sqrt'}


18.)Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?

In [38]:
# Retrieve the best hyperparameters from the RandomizedSearchCV
best_hyperparameters = random_search.best_params_

best_et_classifier = ExtraTreesClassifier(random_state=1, **best_hyperparameters)

# Fit the model on the training data
best_et_classifier.fit(X_train, y_train)

# Evaluate the model on the test data to calculate accuracy
y_pred = best_et_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy of the retrained Extra Trees Classifier:", accuracy)

Accuracy of the retrained Extra Trees Classifier: 0.8041163946061036


20.) Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the two most important respectively?

In [41]:
# Get feature importances from the trained Extra Trees Classifier
feature_importances = best_et_classifier.feature_importances_

# Create a DataFrame to display the feature importances along with their corresponding feature names
importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Print or display the feature importances
print(importances_df)

                                    Feature  Importance
37                  Contract_Month-to-month    0.152237
0                                    tenure    0.092800
19                        OnlineSecurity_No    0.074998
17              InternetService_Fiber optic    0.065287
28                           TechSupport_No    0.064141
39                        Contract_Two year    0.054423
44           PaymentMethod_Electronic check    0.051666
2                              TotalCharges    0.047714
16                      InternetService_DSL    0.032687
22                          OnlineBackup_No    0.030077
38                        Contract_One year    0.028543
21                       OnlineSecurity_Yes    0.021700
25                      DeviceProtection_No    0.016857
1                            MonthlyCharges    0.014926
30                          TechSupport_Yes    0.014559
24                         OnlineBackup_Yes    0.012598
40                      PaperlessBilling_No    0